<a href="https://colab.research.google.com/github/SaeSimcheon/torch/blob/main/Extractive_QA_metric_korea_and_for_korean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from datasets import load_dataset

In [ ]:
data=load_dataset('squad')

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
gold_ex=data['train'][0]['answers']['text'][0]

In [ ]:
pred = gold_ex + '**'

In [ ]:
import re
import string

def normalize_answer(s):
    def remove_punc(text):
        korean_punctuation = '！＂＃＄％＆＇（）＊＋，－．／：；＜＝＞＠［＼］＾＿｀｛｜｝～'  # 한글 특수문자
        all_punctuation = string.punctuation + korean_punctuation
        return ''.join(ch for ch in text if ch not in all_punctuation)

    s = remove_punc(s)  # 구두점 제거
    s = re.sub(r'\s+', ' ', s).strip()  # 연속된 공백은 하나로 통합하고 양쪽 공백 제거
    return s


In [ ]:
def compute_em(pred, gold):
    return int(normalize_answer(pred) == normalize_answer(gold))

In [ ]:
compute_em(gold_ex,pred)

1

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()

# 예시 문장 분석
text = "사과를 먹었다&*^*^@@@"
print(kkma.morphs(text))  # ['사과', '를', '먹', '었', '다']

['사과', '를', '먹', '었', '다', '&', '*', '^', '*', '^', '@@@']


In [ ]:
refined_answer = normalize_answer(text)

In [ ]:
print(kkma.morphs(refined_answer))  # ['사과', '를', '먹', '었', '다']

['사과', '를', '먹', '었', '다']


In [ ]:
def em(pred,answer):
  refined_answer = normalize_answer(answer)
  refined_pred = normalize_answer(pred)

  analyzed_answer =kkma.morphs(refined_answer)
  analyzed_pred =kkma.morphs(refined_pred)

  return int(analyzed_answer == analyzed_pred)

In [ ]:
answer_text = "사과를 먹었다"
pred_text = "사과를 먹었다"
print(f"EM Score: {em(pred_text, answer_text)}")

EM Score: 1


In [ ]:
from collections import Counter
def f1score(pred,answer):
  refined_answer = normalize_answer(answer)
  refined_pred = normalize_answer(pred)

  analyzed_answer =kkma.morphs(refined_answer)
  analyzed_pred =kkma.morphs(refined_pred)

  set_answer=Counter(analyzed_answer)
  set_pred=Counter(analyzed_pred)

  c = set_answer & set_pred
  print(set_answer,set_pred,c)
  num = sum(c.values())

  precision = num / len(analyzed_answer)
  recall = num / len(analyzed_pred)
  f1 = 2 * precision * recall / (precision + recall)
  return f1

In [ ]:
answer_text = "사과를 먹고 또 사과를 먹었다"
pred_text = "사과 먹었다"
print(f"f1 Score: {f1score(pred_text, answer_text)}")

Counter({'사과': 2, '를': 2, '먹': 2, '고': 1, '또': 1, '었': 1, '다': 1}) Counter({'사과': 1, '먹': 1, '었': 1, '다': 1}) Counter({'사과': 1, '먹': 1, '었': 1, '다': 1})
f1 Score: 0.5714285714285715


In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

data = load_dataset("squad_kor_v1")

In [ ]:
import os
import requests
import json
import regex as re

In [ ]:
'''
def char_byte_map(text):

  char_to_byte = [0]*len(text)

  byte_to_char = [0]*len(text.encode('utf-8'))
  byte_index = 0
  for char_index,char in enumerate(text):

    char_to_byte[char_index] = byte_index

    length = len(char.encode('utf-8'))

    for i in range(length):
      byte_to_char[byte_index+i] = char_index

    byte_index+=length

  return char_to_byte,byte_to_char
'''

In [ ]:
def char_byte_map(text):
    char_to_byte = [0] * len(text)
    total_bytes = len(text.encode('utf-8'))
    # byte_to_char 리스트 크기를 total_bytes + 1로 설정
    byte_to_char = [0] * (total_bytes + 1)

    byte_index = 0
    for char_index, char in enumerate(text):
        char_to_byte[char_index] = byte_index

        char_bytes = char.encode('utf-8')
        length = len(char_bytes)

        for i in range(length):
            byte_to_char[byte_index + i] = char_index

        byte_index += length


    byte_to_char[total_bytes] = len(text)

    return char_to_byte, byte_to_char

In [ ]:
def bytes_to_unicode():
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:] # all integers b in bs will simply map to chr(b) in the output dict
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    d = dict(zip(bs, cs))
    return d

In [ ]:
def get_pairs(subwords):
  pairs = set()
  for i in range(len(subwords)-1):
    pair = (subwords[i],subwords[i+1])
    pairs.add(pair)
  return pairs

In [ ]:
def get_file(local_file, remote_file):
    """ downloads remote_file to local_file if necessary """
    if not os.path.isfile(local_file):
        print(f"downloading {remote_file} to {local_file}")
        response = requests.get(remote_file)
        open(local_file, "wb").write(response.content)

def get_encoder():
    """
    Returns an instance of the GPT BPE Encoder/Decoder
    and handles caching of "database" files.
    """
    home_dir = './'



    # load encoder.json that has the raw mappings from token -> bpe index
    encoder_local_file = os.path.join('./', 'encoder.json')
    encoder_remote_file = 'https://huggingface.co/kakaobrain/kogpt/resolve/main/tokenizer/encoder.json'
    get_file(encoder_local_file, encoder_remote_file)
    with open(encoder_local_file, 'r') as f:
        encoder = json.load(f)
    assert len(encoder) == 50257 # 256 individual byte tokens, 50,000 merged tokens, and 1 special <|endoftext|> token

    # load vocab.bpe that contains the bpe merges, i.e. the bpe tree structure
    # in the form tuples (a, b), that indicate that (a, b) is to be merged to one token ab
    vocab_local_file = os.path.join('./', 'vocab.bpe')
    vocab_remote_file = 'https://huggingface.co/kakaobrain/kogpt/resolve/main/tokenizer/vocab.bpe'
    get_file(vocab_local_file, vocab_remote_file)
    with open(vocab_local_file, 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    # light postprocessing: strip the version on first line and the last line is a blank
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    assert len(bpe_merges) == 50000 # 50,000 merged tokens

    # construct the Encoder object and return

    return encoder,bpe_merges

In [ ]:
encoder,bpe_merges=get_encoder()

In [ ]:
class Encoder :
  def __init__(self,encoder,bpe_merge_rule):

    self.byte_encoder = bytes_to_unicode()
    self.byte_decoder = {v:k for k,v in self.byte_encoder.items()}
    self.encoder = encoder
    self.decoder = {v:k for k,v in self.encoder.items()}

    self.bpe_ranks = dict(zip(bpe_merge_rule,range(len(bpe_merge_rule))))

    self.pat = re.compile(r"[\p{IsHangul}]+| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+")
    self.cache = {}

  def bpe(self,unicode_pre_token):

    word = tuple(unicode_pre_token)
    pairs = get_pairs(word)

    while pairs :

      pair = min(pairs,key = lambda pair:self.bpe_ranks.get(pair,float('inf')))
      first,second = pair
      if pair not in self.bpe_ranks :
        break
      new_word = []
      i = 0
      while i < len(word):
        if i < len(word)-1 and word[i] == first and word[i+1] == second:
          new_word.append(word[i]+word[i+1])
          i+=2
        else:
          new_word.append(word[i])
          i+=1

      word = new_word
      pairs = get_pairs(word)

    return ' '.join(word)

  def encoder_with_offsets(self,text):
    output_index = []
    offsets = []
    char_to_byte,byte_to_char = char_byte_map(text)
    for match in re.finditer(self.pat,text):
      pre_token = match.group(0)
      start_char_index_in_text = match.start()
      start_byte_index_in_text = char_to_byte[start_char_index_in_text]
      encoded_pre_token = pre_token.encode('utf-8')

      unicode_pre_token = ''.join([self.byte_encoder[b] for b in encoded_pre_token])
      subwords = self.bpe(unicode_pre_token).split(' ')

      byte_cursor = 0  # encoded_pre_token 기준
      for subword in subwords:
          # subword를 디코딩해서 실제 바이트 길이를 얻음
          subword_bytes = bytes([self.byte_decoder[c] for c in subword])
          subword_len = len(subword_bytes)

          # 전체 text에서 이 subword의 시작 byte index
          global_byte_index = start_byte_index_in_text + byte_cursor

          local_char_index = byte_to_char[global_byte_index]

          char_end_index = byte_to_char[global_byte_index+subword_len] if global_byte_index+subword_len < len(byte_to_char) else len(text)

          offsets.append((local_char_index,char_end_index))
          output_index.append(self.encoder[subword])

          byte_cursor += subword_len  # 다음 subword 위치로 이동
    return output_index,offsets

  def decode(self,index):

    joined_subwords =''.join([self.decoder[i] for i in index])

    decoded = bytearray([self.byte_decoder[c] for c in joined_subwords])


    text=decoded.decode('utf-8',errors = 'replace')

    return text

In [ ]:
enc = Encoder(encoder,bpe_merges)

In [ ]:
!pip install transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer
import torch # return_tensors='pt'를 위해 필요

text = "안녕하세요, extractive QA 테스트입니다."

# 사용할 모델의 이름을 klue/roberta-base 로 변경 ★★★
# use_fast=True 옵션 유지 ★★★
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base", use_fast=True)

# 로드된 토크나이저가 빠른 토크나이저인지 확인해봅니다.
print(f"Is the loaded tokenizer fast? {tokenizer.is_fast}") # True가 나와야 합니다.


# 토큰화 수행 시 offset_mapping=True 옵션을 설정
encoded = tokenizer(text, return_offsets_mapping=True, return_tensors='pt')

# 결과 확인
input_ids = encoded['input_ids']
# KLUE-RoBERTa는 BERT와 달리 token_type_ids를 사용하지 않습니다.
attention_mask = encoded['attention_mask']
offsets = encoded['offset_mapping'] # <-- 이제 이 정보가 정상적으로 반환될 것입니다.

print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)
# print("Token Type IDs:", token_type_ids) # KLUE-RoBERTa에서는 없음
print("Offsets:", offsets)

# 오프셋 정보를 이용해 토큰과 원본 텍스트 매핑 확인
tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

print("\nTokens with Offsets:")
for token, offset in zip(tokens, offsets[0].tolist()):
    if offset is not None:
         original_span = text[offset[0]:offset[1]]
         print(f"'{token}': {offset} -> '{original_span}'")
    else:
         print(f"'{token}': {offset}")

# 예시: 임의의 토큰 인덱스 범위를 선택하여 원본 텍스트 스팬 추출
# (위 print 결과의 토큰/오프셋을 보고 유효한 범위를 지정해주세요)
# 여기서는 예시로 인덱스 5부터 8까지를 사용합니다.
predicted_start_token = 5
predicted_end_token = 8

if 5 < len(tokens) and 8 < len(tokens):
    # 예측된 토큰 인덱스에 해당하는 원본 텍스트 스팬 추출
    # 시작 토큰의 시작 오프셋 (offset[predicted_start_token][0])
    # 끝 토큰의 끝 오프셋 (offset[predicted_end_token][1])
    start_char = offsets[0][predicted_start_token][0].item()
    end_char = offsets[0][predicted_end_token][1].item()

    predicted_answer_span = text[start_char:end_char]
    print(f"\n토큰 인덱스 [{predicted_start_token}, {predicted_end_token}]에 해당하는 원본 텍스트 스팬: '{predicted_answer_span}'")
else:
     print("\n예시 토큰 인덱스 [5, 8]이 유효한 범위를 벗어납니다.")

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Is the loaded tokenizer fast? True
Input IDs: tensor([[    0,  5891,  2205,  5971,    16, 12235, 14766, 16023, 15743,    53,
          2106,  7453, 12190,    18,     2]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Offsets: tensor([[[ 0,  0],
         [ 0,  2],
         [ 2,  3],
         [ 3,  5],
         [ 5,  6],
         [ 7,  9],
         [ 9, 11],
         [11, 14],
         [14, 17],
         [18, 19],
         [19, 20],
         [21, 24],
         [24, 27],
         [27, 28],
         [ 0,  0]]])

Tokens with Offsets:
'[CLS]': [0, 0] -> ''
'안녕': [0, 2] -> '안녕'
'##하': [2, 3] -> '하'
'##세요': [3, 5] -> '세요'
',': [5, 6] -> ','
'ex': [7, 9] -> 'ex'
'##tr': [9, 11] -> 'tr'
'##act': [11, 14] -> 'act'
'##ive': [14, 17] -> 'ive'
'Q': [18, 19] -> 'Q'
'##A': [19, 20] -> 'A'
'테스트': [21, 24] -> '테스트'
'##입니다': [24, 27] -> '입니다'
'.': [27, 28] -> '.'
'[SEP]': [0, 0] -> ''

토큰 인덱스 [5, 8]에 해당하는 원본 텍스트 스팬: 'extractive'


In [ ]:
class config :
  max_length = 256
  batch_size = 32
  hidden_dim = 512
  num_heads = 8
  dropout_rate = 0.1
  ffn_dim = 512*4
  num_layers = 6
  voca_size = tokenizer.vocab_size
  padding_id = tokenizer.pad_token_id
  lr = 1e-5
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer # Hugging Face 토크나이저 사용 가정

class QADataset(Dataset):
    def __init__(self, tokenizer: AutoTokenizer, full_text: list, config, is_training: bool = True):
        self.tokenizer = tokenizer
        self.full_text = full_text
        self.config = config
        self.is_training = is_training
        self.max_length = config.max_length # 모델의 최대 길이

        self.padding_id = self.tokenizer.pad_token_id # 토크나이저의 패딩 ID

        self.features = []


        self.cls_token_id = self.tokenizer.cls_token_id
        self.sep_token_id = self.tokenizer.sep_token_id
        self.pad_token_id = self.tokenizer.pad_token_id

        self._build()

    def _build(self):

        for a_sample in self.full_text:
            q = a_sample['question']
            c = a_sample['context']

            answer_text = a_sample['answers']['text'][0]
            answer_start_in_context = a_sample['answers']['answer_start'][0]
            answer_end_in_context = answer_start_in_context + len(answer_text)

            encoded = self.tokenizer(
                q,
                text_pair=c,
                padding='max_length',
                truncation=True,
                max_length=self.max_length,
                return_offsets_mapping=True,
                return_attention_mask=True,

            )


            input_ids = encoded['input_ids']
            attention_mask = encoded['attention_mask']
            offset_mapping = encoded['offset_mapping']

            second_sep_token_index = -1

            for i,flag in enumerate(encoded.sequence_ids()):
              if flag is not None and flag == 1 :
                second_sep_token_index = i
                break

            context_start_token_index = second_sep_token_index


            start_token_idx = -1
            end_token_idx = -1
            for i in range(self.max_length):
                if offset_mapping[i] == (-1, -1):
                    continue

                token_char_start, token_char_end = offset_mapping[i]
                is_context_token = (i >= context_start_token_index)


                if is_context_token:
                    if token_char_start <= answer_start_in_context < token_char_end:
                         if start_token_idx == -1:
                              start_token_idx = i
                    if token_char_start <= answer_end_in_context - 1 < token_char_end:
                         end_token_idx = i

            is_valid_answer_span = False
            if start_token_idx != -1 and end_token_idx != -1 and start_token_idx <= end_token_idx:
                 predicted_ans_start_char = offset_mapping[start_token_idx][0]
                 predicted_ans_end_char = offset_mapping[end_token_idx][1]
                 extracted_text = c[predicted_ans_start_char:predicted_ans_end_char]

                 if extracted_text == answer_text:
                      is_valid_answer_span = True




            if self.is_training and not is_valid_answer_span:

                 continue



            feature = {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'offset_mapping': offset_mapping,
                'question_id': a_sample['id'],
                'answer_text': answer_text,
                'context': c,
            }
            if self.is_training:
                 if is_valid_answer_span:
                      feature['start_positions'] = start_token_idx
                      feature['end_positions'] = end_token_idx


            self.features.append(feature)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        item = {
            'input_ids': torch.as_tensor(feature['input_ids'], dtype=torch.long),
            'attention_mask': torch.as_tensor(feature['attention_mask'], dtype=torch.long),

            'offset_mapping': torch.as_tensor(feature['offset_mapping'], dtype=torch.long),
            'question_id': feature['question_id'],
            'answer_text': feature['answer_text'], # 검증/디버깅용
            'context': feature['context'] # 검증/디버깅용

        }
        # 학습 시에만 정답 레이블 포함
        if self.is_training:
            item['start_positions'] = torch.as_tensor(feature['start_positions'], dtype=torch.long)
            item['end_positions'] = torch.as_tensor(feature['end_positions'], dtype=torch.long)

        return item

In [ ]:
del data_inst

In [ ]:
data_inst=QADataset(tokenizer,data['train'],config,True)

In [ ]:
torch.save(data_inst.features,'./kosquad_train_features.pt')

In [ ]:
class obs(torch.utils.data.Dataset):
  def __init__(self,config,is_training = True):
    self.is_training =is_training
    self.features = torch.load('./kosquad_train_features.pt')

  def __len__(self):
    return len(self.features)
  def __getitem__(self, idx):

      feature = self.features[idx]
      item = {
          'input_ids': torch.as_tensor(feature['input_ids'], dtype=torch.long),
          'attention_mask': torch.as_tensor(feature['attention_mask'], dtype=torch.long),

          'offset_mapping': torch.as_tensor(feature['offset_mapping'], dtype=torch.long),
          'question_id': feature['question_id'],
          'answer_text': feature['answer_text'],
          'context': feature['context']

      }

      if self.is_training:
          item['start_positions'] = torch.as_tensor(feature['start_positions'], dtype=torch.long)
          item['end_positions'] = torch.as_tensor(feature['end_positions'], dtype=torch.long)

      return item

In [ ]:
gc.collect()

38

In [ ]:
del data_inst

NameError: name 'data_inst' is not defined

In [ ]:
o = obs(config,True)

In [ ]:
loader=torch.utils.data.DataLoader(o,batch_size = config.batch_size,shuffle = True)

In [ ]:
for sample in loader :
  print(sample['input_ids'])
  print(sample['start_positions'])
  print(sample['end_positions'])
  break

tensor([[    0, 10665,  2440,  ...,     1,     1,     1],
        [    0,  3665,  3648,  ...,  7285,  5009,     2],
        [    0, 24185,  2079,  ...,  2079, 18016,     2],
        ...,
        [    0, 10062, 11369,  ...,  2200,  3666,     2],
        [    0,  1421,  2075,  ...,     1,     1,     1],
        [    0,  8673,  2151,  ...,     1,     1,     1]])
tensor([211, 110,  89,  42,  16,  59,  33,  59, 171,  59,  93, 155, 150, 153,
         45, 107,  41, 189, 211,  42,  47, 149,  61,  97, 132, 181, 206, 250,
         70, 187, 146,  75])
tensor([215, 110,  93,  48,  18,  60,  35,  64, 172,  59,  94, 156, 150, 154,
         47, 108,  43, 189, 212,  44,  52, 151,  64,  98, 135, 182, 209, 252,
         70, 189, 149,  77])


In [ ]:
class MultiHeadAttention(torch.nn.Module):
  def __init__(self,config):
    super().__init__()
    self.hidden_dim = config.hidden_dim
    self.num_heads = config.num_heads
    self.a_head_dim = self.hidden_dim // self.num_heads

    self.Linear_Q = torch.nn.Linear(config.hidden_dim,config.hidden_dim)
    self.Linear_K = torch.nn.Linear(config.hidden_dim,config.hidden_dim)
    self.Linear_V = torch.nn.Linear(config.hidden_dim,config.hidden_dim)
    self.dropout = torch.nn.Dropout(config.dropout_rate)
    self.Linear_out = torch.nn.Linear(config.hidden_dim,config.hidden_dim)
    for name, param in self.named_parameters():
      if 'weight' in name:
        torch.nn.init.xavier_uniform_(param)
      elif 'bias' in name:
        torch.nn.init.zeros_(param)

  def forward(self,query,key,value,padding_mask,device):
    B,T,S = query.size(0),query.size(1),key.size(1)
    H,NH,aH = self.hidden_dim,self.num_heads,self.a_head_dim
    #print("query mean/std:", query.mean().item(), query.std().item())
    q = self.Linear_Q(query)
    k = self.Linear_K(key)
    v = self.Linear_V(value)

    q = q.view(B,T,NH,aH).permute(0,2,1,3)
    k = k.view(B,S,NH,aH).permute(0,2,1,3)
    v = v.view(B,S,NH,aH).permute(0,2,1,3)

    qk = torch.matmul(q,k.transpose(3,2)) / math.sqrt(aH)

    #print("qk mean:", qk.mean().item())
    #print("qk std :", qk.std().item())

    # key 마스킹
    key_mask = padding_mask.unsqueeze(1).unsqueeze(2).expand(B, 1, T, S)  # (B, 1, T, S)
    #print(padding_mask.sum(dim = -1))
    qk = qk.masked_fill(key_mask == 0, float('-inf'))

    # query 마스킹은 softmax 후 곱셈으로 (안전하게)
    #query_mask = padding_mask.unsqueeze(1).unsqueeze(3).expand(B, 1, T, 1)  # (B, 1, T, 1)

    mask = torch.zeros((T,S),device = device)
    tmp = torch.ones((T,S),dtype = torch.bool,device = device).tril(diagonal=0)
    mask.masked_fill_(tmp.logical_not(),float('-inf'))

    qk += mask
    invalid_mask = torch.isinf(qk).all(dim=-1)

    #print(torch.sum(qk == float('-inf'),dim = -1))

    #qk = self.dropout(qk)
    alpha = torch.softmax(qk,dim = -1)

    #alpha = alpha * query_mask.float()  # query가 padding인 위치는 attention score를 0으로
    #alpha = alpha * key_mask.float()
    alpha = self.dropout(alpha)
    #print("alpha mean:", alpha.mean().item())
    #print("alpha std:", alpha.std().item())

    scores = torch.matmul(alpha,v)
    scores = scores.permute(0,2,1,3).reshape(B,T,H)

    output = self.Linear_out(scores)

    return output

In [ ]:
class NewGELU(torch.nn.Module):
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

In [ ]:
class DecoderLayer(torch.nn.Module):
  def __init__(self,config):
    super().__init__()
    self.config = config
    self.mha_layer_norm = torch.nn.LayerNorm(config.hidden_dim)
    self.mha = MultiHeadAttention(config)
    self.mha_dropout = torch.nn.Dropout(config.dropout_rate)

    self.ffn_layer_norm = torch.nn.LayerNorm(config.hidden_dim)
    self.ffn = torch.nn.Linear(config.hidden_dim,config.ffn_dim)
    self.act = NewGELU()
    self.rev_ffn = torch.nn.Linear(config.ffn_dim,config.hidden_dim)
    self.ffn_dropout = torch.nn.Dropout(config.dropout_rate)

  def _mha(self,src,padding_mask):
    return self.mha_dropout(self.mha(src,src,src,padding_mask,self.config.device))

  def _ffn(self,mha_output):
    inter = self.act(self.ffn(mha_output))
    return self.ffn_dropout(self.rev_ffn(inter))

  def forward(self,src,padding_mask):
    x = src

    x = self.mha_layer_norm(x + self._mha(x,padding_mask))
    return self.ffn_layer_norm(x+ self._ffn(x))

In [ ]:
def get_modules(module,num_layers):
  return torch.nn.ModuleList([module for _ in range(num_layers)])

In [ ]:
class DecoderBlock(torch.nn.Module):
  def __init__(self,config):
    super().__init__()
    self.layers = get_modules(DecoderLayer(config),config.num_layers)

  def forward(self,src,padding_mask):
    output = src
    for layer in self.layers :
      output = layer(output,padding_mask)
    return output

In [ ]:
class Transformer(torch.nn.Module):
  def __init__(self,config):
    super().__init__()
    self.config = config

    self.embedding = torch.nn.Embedding(config.voca_size,config.hidden_dim)
    self.pos = torch.nn.Embedding(config.max_length,config.hidden_dim)
    self.block = DecoderBlock(config)
    self.Layer_norm = torch.nn.LayerNorm(config.hidden_dim)
    self.Linear_start = torch.nn.Linear(config.hidden_dim,1)
    self.Linear_end = torch.nn.Linear(config.hidden_dim,1)

  def forward(self,src,padding_mask):
    T = src.size(1)
    embedded = self.embedding(src)

    seq=torch.arange(T,dtype = torch.long,device = self.config.device)
    pos_embedding = self.pos(seq)
    embedded_src = embedded + pos_embedding
    output=self.block(embedded_src,padding_mask)
    output = self.Layer_norm(output)
    start_output=self.Linear_start(output)
    end_output=self.Linear_end(output)

    return start_output.squeeze(-1),end_output.squeeze(-1)


In [ ]:
tr=Transformer(config)
tr = tr.to(config.device)

In [ ]:
opt = torch.optim.Adam(tr.parameters(),lr = config.lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 20

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
valid_data=QADataset(tokenizer,data['validation'],config,True)
val_loader=torch.utils.data.DataLoader(valid_data,batch_size = config.batch_size,shuffle = False)

In [ ]:
train_losses = []
val_losses = []

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
feature.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'question_id', 'answer_text', 'context', 'start_positions', 'end_positions'])

In [ ]:
import math

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(epochs):
  train_loss = 0
  loop = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
  tr.train()
  for feature in loop:
    src = feature['input_ids'].to(config.device)
    start = feature['start_positions'].to(config.device)
    end = feature['end_positions'].to(config.device)
    padding_mask = feature['attention_mask'].to(config.device)

    opt.zero_grad()
    start_output,end_output = tr(src,padding_mask)

    start_loss = loss_fn(start_output,start)
    end_loss = loss_fn(end_output,end)

    total_loss = (start_loss + end_loss)/2
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(tr.parameters(), max_norm=1.0)

    opt.step()
    train_loss += total_loss.item()
    loop.set_postfix(loss=total_loss.item())
  avg_train_loss = train_loss / len(loader)
  train_losses.append(avg_train_loss)

  tr.eval()
  val_loss = 0
  with torch.no_grad():
    for feature in val_loader:
      src = feature['input_ids'].to(config.device)
      start = feature['start_positions'].to(config.device)
      end = feature['end_positions'].to(config.device)
      padding_mask = feature['attention_mask'].to(config.device)
      start_output,end_output = tr(src,padding_mask)
      start_loss = loss_fn(start_output,start)
      end_loss = loss_fn(end_output,end)

      total_loss = (start_loss + end_loss)/2
      val_loss += total_loss.item()

  avg_val_loss = val_loss / len(val_loader)
  val_losses.append(avg_val_loss)

  tqdm.write(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")



Epoch 1/20:   0%|          | 0/1677 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 4.9948 | Val Loss: 4.8062


Epoch 2/20:   0%|          | 0/1677 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 4.7229 | Val Loss: 4.6815


Epoch 3/20:   0%|          | 0/1677 [00:00<?, ?it/s]

...

[Epoch 8] Train Loss: 4.0531 | Val Loss: 4.3727


Epoch 9/20:   0%|          | 0/1677 [00:00<?, ?it/s]

[Epoch 9] Train Loss: 3.9551 | Val Loss: 4.3932


Epoch 10/20:   0%|          | 0/1677 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
@torch.no_grad()
def val():
  tr.eval()
  val_losses = 0
  val_output = []
  true = []
  for feature in val_loader:
    src = feature['input_ids'].to(config.device)
    start = feature['start_positions'].to(config.device)
    end = feature['end_positions'].to(config.device)
    padding_mask = feature['attention_mask'].to(config.device)
    start_output,end_output = tr(src,padding_mask)
    val_output.append((start_output,end_output))
    true.append((start,end))
    start_loss = loss_fn(start_output,start)
    end_loss = loss_fn(end_output,end)

    total_loss = (start_loss + end_loss)/2
    val_losses += total_loss.item()

  return (val_losses / len(val_loader),val_output,true)

In [ ]:
loss_out,val_output,true=val()

In [ ]:
loss_out

4.3983801868226795

In [ ]:
i = 0

In [ ]:
while True:
  v,start_index=val_output[i][0].topk(dim=-1,k=1)
  v,end_index=val_output[i][1].topk(dim=-1,k=1)
  print(torch.cat([start_index,end_index,true[i][0].unsqueeze(-1),true[i][1].unsqueeze(-1)],dim = -1))
  i+=1
  break

tensor([[ 20, 116, 194, 196],
        [146, 120, 152, 152],
        [ 22,  71, 208, 210],
        [189, 126, 237, 239],
        [ 31,  40,  59,  60],
        [ 68, 175, 246, 250],
        [ 53,  16,  14,  16],
        [ 39, 167,  91,  94],
        [ 23,  28, 115, 117],
        [ 31,  20,  34,  35],
        [ 56, 110,  91,  94],
        [ 33,  23, 160, 161],
        [179, 202,  59,  59],
        [176,  54, 116, 116],
        [134,  84, 145, 146],
        [ 14,  32,  17,  22],
        [179, 202,  59,  59],
        [ 23,  43, 126, 131],
        [ 17,  35,  17,  18],
        [180,  77, 120, 120],
        [180,  87, 224, 224],
        [ 68,  46,  87,  87],
        [ 71,  80, 173, 174],
        [ 29,  33,  32,  33],
        [ 36,  72,  81,  81],
        [ 22,  58, 192, 196],
        [ 18,  22,  21,  22],
        [ 21, 150, 191, 195],
        [ 37,  26,  89,  89],
        [ 61, 192, 191, 192],
        [ 63,  63, 214, 217],
        [ 58,  28,  68,  77]], device='cuda:0')


In [ ]:
v,start_index=val_output[0][0].topk(dim=-1,k=1)

In [ ]:
v,end_index=val_output[0][1].topk(dim=-1,k=1)

In [ ]:
for idx,batch_output in enumerate(val_output):

  for i in range(config.batch_size):
    print(i)
    print(torch.argmax(batch_output[0][i],dim=-1),torch.argmax(batch_output[1][i],dim=-1))
    print(true[idx][0][i],true[idx][1][i])
    print()
  break